In [40]:
import numpy as np
from scipy.spatial import distance_matrix
from copy import copy
from tqdm import tqdm

In [71]:
def select_next_node_AEL(current_node, destination_node, unvisited_nodes, distance_matrix, threshold=0.7):
    """Algorithm Evolution Using Large Language Model"""
    scores = {}
    for node in unvisited_nodes:
        all_distances = [distance_matrix[node][i] for i in unvisited_nodes if i != node]
        average_distance_to_unvisited = np.mean(all_distances)
        std_dev_distance_to_unvisited = np.std(all_distances)
        score = 0.4 * distance_matrix[current_node][node] - 0.3 * average_distance_to_unvisited + 0.2 * std_dev_distance_to_unvisited - 0.1 * distance_matrix[destination_node][node]
        scores[node] = score
    if min(scores.values()) > threshold:
        next_node = min(unvisited_nodes, key=lambda node: distance_matrix[current_node][node])
    else:
        next_node = min(scores, key=scores.get)
    return next_node

def select_next_node_ReEvo(current_node: int, destination_node: int, unvisited_nodes: set, distance_matrix: np.ndarray) -> int:
    """Select the next node to visit from the unvisited nodes."""
    weights = {'distance_to_current': 0.4, 
               'average_distance_to_unvisited': 0.25, 
               'std_dev_distance_to_unvisited': 0.25, 
               'distance_to_destination': 0.1}
    scores = {}
    for node in unvisited_nodes:
        future_distances = [distance_matrix[node, i] for i in unvisited_nodes if i != node]
        if future_distances:
            average_distance_to_unvisited = sum(future_distances) / len(future_distances)
            std_dev_distance_to_unvisited = (sum((x - average_distance_to_unvisited) ** 2 for x in future_distances) / len(future_distances)) ** 0.5
        else:
            average_distance_to_unvisited = std_dev_distance_to_unvisited = 0
        score = (weights['distance_to_current'] * distance_matrix[current_node, node] -
                 weights['average_distance_to_unvisited'] * average_distance_to_unvisited +
                 weights['std_dev_distance_to_unvisited'] * std_dev_distance_to_unvisited -
                 weights['distance_to_destination'] * distance_matrix[destination_node, node])
        scores[node] = score
    next_node = min(scores, key=scores.get)
    return next_node

def select_next_node_nearest(current_node, destination_node, unvisited_nodes, distance_matrix):
    """Nearest Neighbour"""
    return min(unvisited_nodes, key=lambda node: distance_matrix[current_node][node])

def eval_heuristic(node_positions: np.ndarray, select_next_node) -> float:
    '''
    Generate solution for TSP problem using the GPT-generated heuristic algorithm.
    
    Parameters
    ----------
    node_positions : np.ndarray
        2D array of node positions of shape (problem_size, 2).
    
    Returns
    -------
    obj : float
        The length of the generated tour.
    '''
    problem_size = node_positions.shape[0]
    # calculate distance matrix
    dist_mat = distance_matrix(node_positions, node_positions)
    # set the starting node
    start_node = 0
    solution = [start_node]
    # init unvisited nodes
    unvisited = set(range(problem_size))
    # remove the starting node
    unvisited.remove(start_node)
    # run the heuristic
    for _ in range(problem_size - 1):
        next_node = select_next_node(
            current_node=solution[-1],
            destination_node=start_node,
            unvisited_nodes=unvisited,
            distance_matrix=dist_mat,
        )
        solution.append(next_node)
        if next_node in unvisited:
            unvisited.remove(next_node)
        else:
            raise KeyError(f"Node {next_node} is already visited.")
    
    # calculate the length of the tour
    obj = 0
    for i in range(problem_size):
        obj += dist_mat[solution[i], solution[(i + 1) % problem_size]]
    return obj
    

In [72]:
for size in [20, 50, 100, 200, 500, 1000]:
    # Load dataset
    X = np.load('../dataset/test{}_dataset.npy'.format(size))
    objs = []
    print("Evaluating heuristic for size {} with {} instances".format(size, len(X)))
    for node_positions in tqdm(X):
        obj = eval_heuristic(node_positions, select_next_node_nearest)
        objs.append(obj)
    print('Average objective value for size {}: {}'.format(size, np.mean(objs)), "\n")

Evaluating heuristic for size 20 with 64 instances


100%|██████████| 64/64 [00:00<00:00, 13567.62it/s]


Average objective value for size 20: 4.446566634076893 

Evaluating heuristic for size 50 with 64 instances


100%|██████████| 64/64 [00:00<00:00, 4014.29it/s]


Average objective value for size 50: 6.891886602053466 

Evaluating heuristic for size 100 with 64 instances


...

Average objective value for size 200: 13.424787407459862 

Evaluating heuristic for size 500 with 64 instances


100%|██████████| 64/64 [00:01<00:00, 50.19it/s]


Average objective value for size 500: 20.65273146870541 

Evaluating heuristic for size 1000 with 64 instances


100%|██████████| 64/64 [00:04<00:00, 12.94it/s]

Average objective value for size 1000: 29.178004049667557 



In [69]:
for size in [20, 50, 100, 200, 500, 1000]:
    # Load dataset
    X = np.load('../dataset/test{}_dataset.npy'.format(size))
    objs = []
    print("Evaluating heuristic for size {} with {} instances".format(size, len(X)))
    for node_positions in tqdm(X):
        obj = eval_heuristic(node_positions, select_next_node_AEL)
        objs.append(obj)
    print('Average objective value for size {}: {}'.format(size, np.mean(objs)), "\n")

Evaluating heuristic for size 20 with 64 instances


  0%|          | 0/64 [00:00<?, ?it/s]/Users/yhr/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/yhr/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/yhr/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/yhr/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/yhr/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 64/64 [00:00<00:00, 403.66it/s]


Average objective value for size 20: 4.078671740854158 

Evaluating heuristic for size 50 with 64 instances


100%|██████████| 64/64 [00:01<00:00, 52.24it/s]


Average objective value for size 50: 6.233106835234127 

Evaluating heuristic for size 100 with 64 instances


...

Average objective value for size 200: 12.307232622768458 

Evaluating heuristic for size 500 with 64 instances


100%|██████████| 64/64 [07:55<00:00,  7.42s/it]


Average objective value for size 500: 19.23704059951398 

Evaluating heuristic for size 1000 with 64 instances


100%|██████████| 64/64 [56:58<00:00, 53.41s/it]

Average objective value for size 1000: 27.34434531143195 



In [68]:
for size in [20, 50, 100, 200, 500, 1000]:
    # Load dataset
    X = np.load('../dataset/test{}_dataset.npy'.format(size))
    objs = []
    print("Evaluating heuristic for size {} with {} instances".format(size, len(X)))
    for node_positions in tqdm(X):
        obj = eval_heuristic(node_positions, select_next_node_ReEvo)
        objs.append(obj)
    print('Average objective value for size {}: {}'.format(size, np.mean(objs)), "\n")

Evaluating heuristic for size 20 with 64 instances


100%|██████████| 64/64 [00:00<00:00, 1328.55it/s]


Average objective value for size 20: 4.090583020412613 

Evaluating heuristic for size 50 with 64 instances


100%|██████████| 64/64 [00:00<00:00, 114.89it/s]


Average objective value for size 50: 6.2268621650994955 

Evaluating heuristic for size 100 with 64 instances


...

Average objective value for size 200: 12.092696980562213 

Evaluating heuristic for size 500 with 64 instances


100%|██████████| 64/64 [08:22<00:00,  7.86s/it]


Average objective value for size 500: 18.948248117331183 

Evaluating heuristic for size 1000 with 64 instances


100%|██████████| 64/64 [1:06:01<00:00, 61.90s/it]

Average objective value for size 1000: 26.79167114037522 

